In [ ]:
import cv2
import torch
import easyocr
import pytesseract
from ultralytics import YOLO
import pandas as pd
from datetime import datetime
import torchvision.transforms as T
from PIL import Image

In [ ]:
# ================================
# CARGA DE MODELOS
# ================================

device = "cuda" if torch.cuda.is_available() else "cpu"

# 🔹 Detector YOLO entrenado
detector = YOLO("yolo11n_best.pt")

# 🔹 EasyOCR (idiomas español e inglés)
reader_easy = easyocr.Reader(['es', 'en'])

# 🔹 PyTesseract (ruta de instalación local)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# ================================
# CARGA DE MODELO CRNN
# ================================

class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d((2,1), (2,1)),  # conserva ancho
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d((2,1), (2,1))
        )

        # input_size = 512 * (altura reducida)
        self.rnn = nn.LSTM(512, 256, num_layers=2, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.view(b, c*h, w).permute(0, 2, 1)
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x


device = "cuda" if torch.cuda.is_available() else "cpu"

# 🔹 Cargar modelo completo
model_crnn = torch.load("ocr_ctc_full.pt", map_location=device)
model_crnn.eval()
print("✅ Modelo CRNN cargado correctamente.")

# 🔹 CRNN personalizado (modelo completo exportado desde Kaggle)
model_crnn = torch.load("ocr_ctc_full.pt", map_location=device, weights_only=False)
model_crnn.eval()
print("✅ Modelo CRNN cargado correctamente.")

# Transformaciones para CRNN
transform = T.Compose([
    T.Grayscale(),
    T.Resize((32, 128)),
    T.ToTensor(),
])

# Diccionario de caracteres
CHARS = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
idx_to_char = {i: c for i, c in enumerate(CHARS)}

def decode_ctc(output):
    """Decodifica la salida CTC en texto."""
    pred = output.softmax(2).argmax(2).squeeze(0).cpu().numpy()
    text = ""
    prev_char = -1
    for c in pred:
        if c != prev_char and c < len(CHARS):
            text += idx_to_char.get(c, "")
        prev_char = c
    return text

# ================================
# PROCESAMIENTO DE VIDEO
# ================================

VIDEO = "plates_test.mp4"
cap = cv2.VideoCapture(VIDEO)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = 0
data_rows = []

print("Procesando vídeo...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    timestamp = datetime.fromtimestamp((frame_count / fps)).strftime("%H:%M:%S.%f")[:-3]
    results = detector(frame, verbose=False)

    if results[0].boxes:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            placa = frame[y1:y2, x1:x2]
            if placa.size == 0:
                continue

            # EASY OCR
            try:
                text_easy = reader_easy.readtext(placa, detail=0, allowlist=CHARS)
                text_easy = max(text_easy, key=len).replace(" ", "") if text_easy else ""
            except:
                text_easy = ""

            # PYTESSERACT
            try:
                text_tess = pytesseract.image_to_string(
                    placa,
                    config="--psm 8 -c tessedit_char_whitelist=" + CHARS
                )
                text_tess = text_tess.strip().replace(" ", "")
            except:
                text_tess = ""

            # CRNN (nuestro modelo)
            try:
                placa_rgb = cv2.cvtColor(placa, cv2.COLOR_BGR2RGB)
                img_pil = Image.fromarray(placa_rgb)
                img_t = transform(img_pil).unsqueeze(0).to(device)
                with torch.no_grad():
                    out = model_crnn(img_t)
                text_crnn = decode_ctc(out)
            except Exception as e:
                text_crnn = ""

            data_rows.append({
                "Frame": frame_count,
                "Tiempo": timestamp,
                "EasyOCR": text_easy,
                "Tesseract": text_tess,
                "CRNN_Custom": text_crnn
            })

cap.release()
cv2.destroyAllWindows()

# ================================
# GUARDAR RESULTADOS
# ================================

df = pd.DataFrame(data_rows)
df.to_csv("resultados_comparacion.csv", index=False)
print("✅ Comparación completada. Resultados guardados en 'resultados_comparacion.csv'")


C:\Users\Mohamed O. Haroun\AppData\Local\Temp\ipykernel_22420\1046411977.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_crnn = torch.load("ocr_ctc_full.pt", map_

✅ Modelo CRNN cargado correctamente.
✅ Modelo CRNN cargado correctamente.
🔍 Procesando vídeo...


c:\Users\Mohamed O. Haroun\miniconda3\envs\VC_P4\lib\site-packages\torch\nn\modules\rnn.py:1123: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\RNN.cpp:1410.)
  result = _VF.lstm(


✅ Comparación completada. Resultados guardados en 'resultados_comparacion.csv'
